In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

In [2]:
news = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/data/train/RedditNews_train.csv')
labels = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/data/train/Combined_News_DJIA_train.csv')

In [3]:
labels

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0.0,"b""Georgia 'downs two Russian warplanes' as cou...",NaN,b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1.0,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',NaN
2,2008-08-12,0.0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0.0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""",NaN,b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1.0,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Bank analyst forecast Georgian crisis

In [4]:
news.dropna(inplace=True)

In [5]:
news.head()

,Date,News
0,2008-06-08,"b'Marriage, they said, was reduced to the stat..."
1,2008-06-08,b'Nim Chimpsky: The tragedy of the chimp who t...
2,2008-06-08,"b""Canada: Beware slippery slope' to censorship..."
3,2008-06-08,b'EU Vice-President Luisa Morgantini and the I...
6,2008-06-08,b'Chiapas: army occupies Zapatista communities...


In [6]:
news['News'] = news['News'].apply(lambda x: x.lower())

In [7]:
news = news.groupby(['Date'])['News'].apply(lambda x: ', '.join(x)).reset_index()

In [8]:
corpus = news['News'].values
vectorizer = TfidfVectorizer(stop_words='english', max_features=200)
news_vectors = vectorizer.fit_transform(corpus).toarray()
text_features = pd.DataFrame(data=news_vectors,columns=vectorizer.get_feature_names())

In [9]:
text_features = pd.concat([pd.Series(news.Date.unique()).to_frame(name='Date'), text_features], axis=1)

In [10]:
text_features.head()

,Date,000,10,100,20,accused,afghan,afghanistan,africa,aid,...,weapons,week,west,wikileaks,woman,women,workers,world,year,years
0,2008-06-08,0.087215,0.000000,0.0,0.0,0.0,0.000000,0.119453,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.106088,0.0,0.072767,0.000000,0.088168
1,2008-06-09,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.097870,0.0,0.000000,0.000000,0.000000
2,2008-06-10,0.000000,0.000000,0.0,0.0,0.0,0.128819,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.068816,0.241812,0.000000
3,2008-06-11,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.145991,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.158046,0.092559,0.000000
4,2008-06-12,0.000000,0.122114,0.0,0.0,0.0,0.000000,0.119103,0.000000,0.290131,...,0.0,0.0,0.0,0.0,0.0,0.105776,0.0,0.000000,0.000000,0.000000


In [11]:
export = labels[['Date', 'Label']]

In [12]:
export.to_csv('interim_labels.csv', index=False)

In [13]:
data = text_features.merge(export, how='inner', left_on='Date', right_on='Date')

In [14]:
data

,Date,000,10,100,20,accused,afghan,afghanistan,africa,aid,...,week,west,wikileaks,woman,women,workers,world,year,years,Label
0,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.108569,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.115997,0.067933,0.070274,0.0
1,2008-08-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.067818,0.000000,0.000000,1.0
2,2008-08-12,0.000000,0.000000,0.115408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.108559,0.0,0.000000,0.089403,0.000000,0.000000,0.143655,0.000000,0.0
3,2008-08-13,0.000000,0.111657,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.132643,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.077704,0.000000,0.0
4,2008-08-14,0.092446,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154219,...,0.000000,0.000000,0.0,0.000000,0.000000,0.145029,0.077131,0.000000,0.000000,1.0
5,2008-08-15,0.000000,0.000000,0.000000,0.000000,0.171378,0.000000,0.000000,0.000000,0.000000,...,0.175796,0.161927,0.0,0.000000,0.000000,0.000000,0.000000,0.107138,0.000000,1.0
6,2008-08-18,0.118373,0.000000,0.000000,0.184551,0.000000,0.184878,0.000000,0.000000,0.000000,...,0.000000,0.174838,0.0,0.000000,0.143988,0.000000,0.000000,0.115681,0.000000,0.0
7,2008-08-19,0.082708,0.348436,0.000000,0.000000,0.000000,0.129176,0.000000,0.000000,0.137975,...,0.000000,0.000000,0.0,0.113118,0.000000,0.129753,0.000000,0.080827,0.167225,0.0
8,2008-08-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111229,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.197566,0.000000,0.067757,0.079363,0.000000,1.0
9,2008-08-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.139665,0.000000,0.0,0.000000,0.000000,0.000000,0.072671,0.000000,0.000000,1.0


In [15]:
lmtzr.lemmatize('Man goes berzerk in Akihabara and stabs everyone.')

NameError: name 'lmtzr' is not defined

In [ ]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lmtzr.lemmatize(word, tag))
            
    return " ".join(res_words)

In [ ]:
lmtzr = WordNetLemmatizer()

news['News'] = news['News'].apply(lemmatize_sentence)

In [ ]:
lemmatize_sentence('United States quits Human Rights Council')

In [18]:
def sentiment(row):
    return TextBlob(row).sentiment.polarity

def subjectivity(row):
    return TextBlob(row).sentiment.subjectivity

news['sentiment'] = news['News'].apply(sentiment)
news['subjectivity'] = news['News'].apply(subjectivity)

In [19]:
news

,Date,News,sentiment,subjectivity
0,2008-06-08,"b'marriage, they said, was reduced to the stat...",-0.016230,0.419726
1,2008-06-09,"b'chew qat: in yemen, 72 per cent of men and 3...",0.046151,0.359926
2,2008-06-10,b'31 year old beats 3 year old to death: worst...,-0.098525,0.311643
3,2008-06-11,"b""pakistan blames u.s. coalition for troops' d...",-0.065341,0.462500
4,2008-06-12,"b'marine ""mean motari"" gets expelled, another ...",0.032421,0.374246
5,2008-06-13,b'fallujah babies suffer unprecedented deformi...,0.122552,0.454983
6,2008-06-14,"b'""we don\'t have a clue where he is or even m...",0.074451,0.387827
7,2008-06-15,b'pair of reporters interview 66 former guanta...,0.127657,0.380374
8,2008-06-16,b'red alert: talibans on the offensive have ta...,0.091388,0.553947
9,2008-06-17,"b'fifth severed foot found on canadian coast',...",0.129413,0.454103


In [20]:
pd.concat([news[['sentiment','subjectivity']],pd.Series(news.Date.unique()).to_frame(name='Date'), text_features], axis=1)

,sentiment,subjectivity,Date,Date,000,10,100,20,accused,afghan,...,weapons,week,west,wikileaks,woman,women,workers,world,year,years
0,-0.016230,0.419726,2008-06-08,2008-06-08,0.087215,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.106088,0.000000,0.072767,0.000000,0.088168
1,0.046151,0.359926,2008-06-09,2008-06-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.097870,0.000000,0.000000,0.000000,0.000000
2,-0.098525,0.311643,2008-06-10,2008-06-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.128819,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.068816,0.241812,0.000000
3,-0.065341,0.462500,2008-06-11,2008-06-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.158046,0.092559,0.000000
4,0.032421,0.374246,2008-06-12,2008-06-12,0.000000,0.122114,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.105776,0.000000,0.000000,0.000000,0.000000
5,0.122552,0.454983,2008-06-13,2008-06-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.113492,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077679
6,0.074451,0.387827,2008-06-14,2008-06-14,0.085786,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.313049,0.000000,0.071575,0.083835,0.086724
7,0.127657,0.380374,2008-06-15,2008-06-15,0.084017,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.134224,0.000000,0.000000,0.00000,0.114908,0.000000,0.000000,0.210298,0.000000,0.000000
8,0.091388,0.553947,2008-06-16,2008-06-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.180586,0.000000,0.000000
9,0.129413,0.454103,2008-06-17,2008-06-17,0.096234,0.000000,0.000000,0.150034,0.000000,0.000000,...,0.000000,0.000000,0.142138,0.00000,0.000000,0.000000,0.000000,0.321167,0.000000,0.000000


In [21]:
news[['sentiment','subjectivity']]

,sentiment,subjectivity
0,-0.016230,0.419726
1,0.046151,0.359926
2,-0.098525,0.311643
3,-0.065341,0.462500
4,0.032421,0.374246
5,0.122552,0.454983
6,0.074451,0.387827
7,0.127657,0.380374
8,0.091388,0.553947
9,0.129413,0.454103
